In [1]:
# Importamos las librerías necesarias
import findspark
findspark.init() # Asegurémonos de que encuentre Spark

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, date_trunc
from pyspark.sql.types import StringType, BooleanType, TimestampType, DoubleType, IntegerType, LongType # Tipos necesarios
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, MinMaxScaler as SparkMinMaxScaler, StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.functions import vector_to_array # Útil para ver vectores

from sklearn.preprocessing import MinMaxScaler as SklearnMinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.layers import (Input, Dense, Dropout, LayerNormalization,
                                     MultiHeadAttention, GlobalAveragePooling1D, Add)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

print("Librerías importadas correctamente.")

2025-05-07 21:18:20.108212: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-07 21:18:20.118317: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-07 21:18:20.128840: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-07 21:18:20.132083: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-07 21:18:20.141010: I tensorflow/core/platform/cpu_feature_guar

Librerías importadas correctamente.


In [2]:
# --- 1. Configuración de Spark ---
# Configuramos e iniciamos nuestra sesión de Spark en modo local.
# Usamos rutas absolutas para los directorios de warehouse y metastore para evitar problemas.
print("Configurando Spark Session...")
spark = SparkSession.builder \
    .appName("PrediccionTiempoEsperaAvionNotebook") \
    .config("spark.master", "local[*]") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .config("spark.sql.warehouse.dir", f"file:///{os.path.abspath('spark-warehouse')}") \
    .config("spark.driver.extraJavaOptions", f"-Dderby.system.home={os.path.abspath('derby_metastore_db')}") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

sc = spark.sparkContext
print(f"Spark Session creada. Default FS: {spark.conf.get('spark.hadoop.fs.defaultFS')}")
print(f"Usando optimización Arrow: {spark.conf.get('spark.sql.execution.arrow.pyspark.enabled')}")

# Mostramos la interfaz de usuario de Spark si está disponible (útil para monitorear)
print(f"Spark UI disponible en: {sc.uiWebUrl}")

Configurando Spark Session...


25/05/07 21:18:23 WARN Utils: Your hostname, bryanSpace resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/07 21:18:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 21:18:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Session creada. Default FS: file:///
Usando optimización Arrow: true
Spark UI disponible en: http://10.255.255.254:4041


In [3]:
# --- 2. Definición de Rutas y Carga de Datos ---
# Entrenaremos el modelo con todos los datos
BASE_DATA_PATH = "despegues/src/dato/completo" # Ajusta si es necesario
TRAIN_DATA_PATH = os.path.join(BASE_DATA_PATH, "*.parquet") # <- Cambia a tu ruta real
METEO_DATA_PATH = os.path.join(BASE_DATA_PATH, "open-meteo-40.53N3.56W602m-2.csv") # <- Cambia a tu ruta real

print(f"Ruta base de datos: {BASE_DATA_PATH}")
print(f"Cargando datos de entrenamiento desde: {TRAIN_DATA_PATH}")
df_train = spark.read.parquet(TRAIN_DATA_PATH)


print(f"\nSe cargaron {df_train.count()} registros de entrenamiento.")

Ruta base de datos: despegues/src/dato/completo
Cargando datos de entrenamiento desde: despegues/src/dato/completo/*.parquet

Se cargaron 151524 registros de entrenamiento.


In [4]:
# --- 3. Función para añadir datos meteorológicos ---
# Definimos una función reutilizable para cargar el CSV de meteo, procesarlo
# y unirlo a un DataFrame de Spark basado en la hora truncada del timestamp.
from pyspark.sql.functions import col, date_trunc, from_unixtime


def add_meteo_data(spark_df, meteo_csv_path):
    """Carga datos meteorológicos y los une a un DataFrame Spark."""
    print(f"Cargando datos meteorológicos desde: {meteo_csv_path}")
    # Usamos Pandas para leer el CSV por comodidad con skiprows, luego convertimos a Spark DF
    pdf_meteo = pd.read_csv(meteo_csv_path, skiprows=2)
    df_meteo = spark.createDataFrame(pdf_meteo)

    # Convertimos la columna 'time' (string) a timestamp y la renombramos para la unión
    df_meteo = df_meteo.withColumn("time_hour_meteo", to_timestamp(col("time")))
    df_meteo = df_meteo.drop("time") # Eliminamos la columna original 'time' string

    print("Procesando timestamps en el DataFrame principal para la unión...")
    # Creamos una columna truncando el timestamp principal a la hora en punto
    spark_df = spark_df.withColumn("time_hour", date_trunc("hour", col("timestamp")))

    # Unimos los DataFrames por la hora truncada
    print("Uniendo datos principales con datos meteorológicos (join)...")
    # Usamos un left join por si algún registro principal no tiene correspondencia exacta de hora en meteo
    # Aunque con datos horarios, un inner join debería funcionar bien si los rangos coinciden.
    # Cambiamos a inner join como en el código original, asumiendo cobertura completa.

        # Antes del join, elimina del df_train las columnas que también existen en el CSV meteo
    meteo_cols = [c for c in df_meteo.columns if c != "time_hour_meteo"]
    cols_to_drop = [c for c in spark_df.columns if c in meteo_cols]

    if cols_to_drop:
        print(f"Eliminando {len(cols_to_drop)} columnas duplicadas del df principal: {cols_to_drop}")
        spark_df = spark_df.drop(*cols_to_drop)
    joined_df = spark_df.join(df_meteo, spark_df["time_hour"] == df_meteo["time_hour_meteo"], how="inner")

    # Eliminamos las columnas de tiempo intermedias usadas para la unión
    joined_df = joined_df.drop("time_hour", "time_hour_meteo")
    print(f"Unión completada.")
    return joined_df

# Ahora aplicamos la función a nuestros DataFrames de entrenamiento y prueba
print("\nAplicando unión meteorológica a datos de entrenamiento...")
df_train = add_meteo_data(df_train, METEO_DATA_PATH)


Aplicando unión meteorológica a datos de entrenamiento...
Cargando datos meteorológicos desde: despegues/src/dato/completo/open-meteo-40.53N3.56W602m-2.csv


/home/bryan/pruebas/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:351: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


Procesando timestamps en el DataFrame principal para la unión...
Uniendo datos principales con datos meteorológicos (join)...
Eliminando 19 columnas duplicadas del df principal: ['temperature_2m (°C)', 'relative_humidity_2m (%)', 'dew_point_2m (°C)', 'precipitation (mm)', 'snowfall (cm)', 'weather_code (wmo code)', 'surface_pressure (hPa)', 'cloud_cover (%)', 'cloud_cover_low (%)', 'cloud_cover_mid (%)', 'cloud_cover_high (%)', 'is_day ()', 'wind_speed_10m (km/h)', 'wind_direction_10m (°)', 'wind_direction_100m (°)', 'soil_moisture_0_to_7cm (m³/m³)', 'soil_temperature_100_to_255cm (°C)', 'soil_moisture_100_to_255cm (m³/m³)', 'et0_fao_evapotranspiration (mm)']
Unión completada.


In [5]:
from pyspark.sql.functions import log1p, col

print("Aplicando transformación log(1+x) a takeoff_time...")
df_train = df_train.withColumn("log_takeoff_time", log1p(col("takeoff_time")))

TARGET_COL = "log_takeoff_time"
print(f"La nueva columna objetivo es: {TARGET_COL}")

Aplicando transformación log(1+x) a takeoff_time...
La nueva columna objetivo es: log_takeoff_time


In [6]:
# Celda Corregida para Ingeniería de Características Cíclicas

from pyspark.sql.functions import sin, cos, pi, col, lit, when, lower

print("Creando características cíclicas para hora y día de la semana...")

# --- 1. Mapear 'weekday' (string) a 'weekday_num' (0-6) ---
# Ajusta el mapeo según tus datos (si Lunes es 0 o Domingo es 0, etc.)
# Usamos lower() para ser insensible a mayúsculas/minúsculas
weekday_mapping = {
    "mon": 0, "tue": 1, "wed": 2, "thu": 3, "fri": 4, "sat": 5, "sun": 6
}

numerical_cols = []
categorical_cols = []

# Creamos la expresión 'when' para el mapeo
mapping_expr = None
for day_str, day_num in weekday_mapping.items():
    if mapping_expr is None:
        mapping_expr = when(lower(col("weekday")) == day_str, day_num)
    else:
        mapping_expr = mapping_expr.when(lower(col("weekday")) == day_str, day_num)
# Añadimos un 'otherwise' por si hay algún valor inesperado (opcional, podrías poner null o un valor por defecto)
mapping_expr = mapping_expr.otherwise(None) # O por ejemplo .otherwise(0)

# Aplicamos el mapeo a df_train y df_test
df_train = df_train.withColumn("weekday_num", mapping_expr)

# --- 2. Crear Características Cíclicas ---
# Para hora (0-23) -> hour_sin, hour_cos
df_train = df_train.withColumn("hour_sin", sin(lit(2) * pi() * col("hour") / lit(24.0)))
df_train = df_train.withColumn("hour_cos", cos(lit(2) * pi() * col("hour") / lit(24.0)))

# Para día de la semana (0-6) -> weekday_sin, weekday_cos
df_train = df_train.withColumn("weekday_sin", sin(lit(2) * pi() * col("weekday_num") / lit(7.0)))
df_train = df_train.withColumn("weekday_cos", cos(lit(2) * pi() * col("weekday_num") / lit(7.0)))

print("Características cíclicas creadas.")

# --- 3. Actualizar Listas de Features ---
# Asumiendo que las listas numerical_cols y categorical_cols ya existen de la celda anterior

# Definimos las nuevas columnas y las que vamos a quitar
new_cyclical_cols = ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos']
cols_to_remove = ['hour', 'weekday', 'weekday_num'] # Quitamos hour, weekday(string), y weekday_num(intermedia)

# Añadimos las nuevas columnas cíclicas a las numéricas
numerical_cols.extend([c for c in new_cyclical_cols if c not in numerical_cols])

# Eliminamos las columnas originales/intermedias de las listas
print(f"Eliminando columnas originales/intermedias: {cols_to_remove} de las listas de features")
numerical_cols = [c for c in numerical_cols if c not in cols_to_remove]
categorical_cols = [c for c in categorical_cols if c not in cols_to_remove]

# Actualizamos la lista general de features (útil para verificar)
feature_cols = numerical_cols + categorical_cols
print(f"Nuevas columnas numéricas: {numerical_cols}")
print(f"Nuevas columnas categóricas: {categorical_cols}")
print(f"Total features actualizado: {len(feature_cols)}")

# Verificamos que las nuevas columnas existen y las viejas no (opcional)
print("\nColumnas después de añadir cíclicas y eliminar originales (primeras 5):")
print(df_train.columns[:5]) # Muestra solo las primeras para brevedad
# Deberías ver 'hour_sin', 'hour_cos', etc. más adelante y no 'hour', 'weekday'

Creando características cíclicas para hora y día de la semana...
Características cíclicas creadas.
Eliminando columnas originales/intermedias: ['hour', 'weekday', 'weekday_num'] de las listas de features
Nuevas columnas numéricas: ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos']
Nuevas columnas categóricas: []
Total features actualizado: 4

Columnas después de añadir cíclicas y eliminar originales (primeras 5):
['takeoff_time', 'timestamp', 'icao', 'callsign', 'holding_point']


In [7]:
# --- 4. Preparación de Características (Spark ML Pipeline) ---


# Identificamos las columnas que NO usaremos como características (features).
# Incluimos identificadores, timestamps brutos y la propia columna objetivo.
excluded_cols = [TARGET_COL, 'timestamp', 'icao', 'callsign', 'event_timestamp',
                 'first_holding_time', 'first_airborne_time', 'first_on_ground_time']

# El resto de columnas serán nuestras características iniciales.
# Obtenemos la lista de columnas del DataFrame de entrenamiento (ya incluye las de meteo)
all_cols = df_train.columns
feature_cols = [c for c in all_cols if c not in excluded_cols]
# Asegúrate de que la columna original 'takeoff_time' NO esté en feature_cols
if 'takeoff_time' in numerical_cols: numerical_cols.remove('takeoff_time')
if 'takeoff_time' in feature_cols: feature_cols.remove('takeoff_time')
    
# Separamos las características en categóricas y numéricas/booleanas para el pipeline.
# Lo hacemos basándonos en el esquema del DataFrame de entrenamiento.


print("\nIdentificando tipos de columnas para el pipeline...")
for col_name in feature_cols:
    dtype = df_train.schema[col_name].dataType
    # print(f" - Columna: {col_name}, Tipo: {dtype}") # Descomentar para depuración detallada
    if isinstance(dtype, StringType):
        categorical_cols.append(col_name)
    # Tratamos Booleanos como numéricos (Spark los manejará como 0/1 en VectorAssembler)
    elif isinstance(dtype, (DoubleType, IntegerType, LongType, BooleanType)):
         numerical_cols.append(col_name)
    # Advertimos si encontramos Timestamps como features, ya que podrían necesitar tratamiento especial
    elif isinstance(dtype, TimestampType):
         print(f"    ADVERTENCIA: La columna Timestamp '{col_name}' está en las features. Se tratará como numérica (epoch). Considera extraer features de tiempo (hora, día, etc.) si es relevante.")
         # La convertiremos a número (segundos epoch) antes del ensamblador si es necesario,
         # pero VectorAssembler podría manejarlo implícitamente en versiones recientes.
         # Por seguridad, la añadimos a numéricas.
         numerical_cols.append(col_name)
    else:
         print(f"    ADVERTENCIA: Tipo de dato no manejado directamente: {dtype} para columna '{col_name}'. Se omitirá.")

# --- AÑADIR ESTE BLOQUE DE IMPUTACIÓN ---
print("\nImputando valores nulos/NaN en df_train ANTES del pipeline...")

# Estrategia: Rellenar numéricos con 0 y categóricos con un placeholder string.
# (Podrías usar otras estrategias como media/mediana para numéricos con Spark ML Imputer si lo prefieres)

# Rellenar NaNs en columnas numéricas con 0
# Asegúrate de que 'numerical_cols' contiene los nombres correctos
print(f"Rellenando NaNs con 0 en {len(numerical_cols)} columnas numéricas/booleanas...")
df_train = df_train.na.fill(0, subset=numerical_cols)

# Rellenar NaNs/Nulls en columnas categóricas con un string específico
# Asegúrate de que 'categorical_cols' contiene los nombres correctos
placeholder_string = "__MISSING__" # O cualquier string que no exista en tus datos
print(f"Rellenando Nulls con '{placeholder_string}' en {len(categorical_cols)} columnas categóricas...")
df_train = df_train.na.fill(placeholder_string, subset=categorical_cols)

print("Imputación completada en df_train.")
# --- FIN DEL BLOQUE DE IMPUTACIÓN ---

# --- Ahora continúa con la definición del pipeline como antes ---
# 1. StringIndexers (como antes)
indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid='keep')
    for col in categorical_cols
]
print(f"\nDefinidos {len(indexers)} StringIndexers.")

# 2. OneHotEncoder
# Las columnas de salida de StringIndexer serán las de entrada para OneHotEncoder
ohe_stages = [
    OneHotEncoder(inputCol=f"{col}_index", outputCol=f"{col}_ohe")
    for col in categorical_cols
]
print(f"Definidos {len(ohe_stages)} OneHotEncoders.")

# 3. VectorAssembler (actualizado para usar las columnas OHE)
# Las columnas de entrada ahora serán las numéricas y las nuevas columnas '_ohe'
ohe_output_cols = [f"{c}_ohe" for c in categorical_cols]
assembler_input_cols = ohe_output_cols + numerical_cols

assembler = VectorAssembler(
    inputCols=assembler_input_cols,
    outputCol="features_raw",
    handleInvalid='keep' # o 'skip' si prefieres no tener un vector extra grande si hay un error
)
print(f"Definido VectorAssembler con {len(assembler_input_cols)} columnas de entrada (usando OHE).")

# 4. Scaler (como antes, pero actuará sobre el vector que incluye OHE)
scaler = SparkMinMaxScaler(inputCol="features_raw", outputCol="features")
print("Definido SparkMinMaxScaler.")

# 5. Pipeline (actualizado para incluir los OHE)
pipeline = Pipeline(stages=indexers + ohe_stages + [assembler, scaler]) # Añadir ohe_stages
print("Pipeline de preprocesamiento (con OHE) definido.")


Identificando tipos de columnas para el pipeline...

Imputando valores nulos/NaN en df_train ANTES del pipeline...
Rellenando NaNs con 0 en 73 columnas numéricas/booleanas...
Rellenando Nulls con '__MISSING__' en 6 columnas categóricas...
Imputación completada en df_train.

Definidos 6 StringIndexers.
Definidos 6 OneHotEncoders.
Definido VectorAssembler con 79 columnas de entrada (usando OHE).
Definido SparkMinMaxScaler.
Pipeline de preprocesamiento (con OHE) definido.


In [9]:
# --- Ajuste (Fit) del Pipeline ---
# Ajustamos el pipeline COMPLETO (indexers, assembler, scaler) usando SÓLO los datos de entrenamiento.
# Spark aprenderá los índices de las categorías, las estadísticas para el escalado, etc., a partir de df_train.
print("\nAjustando el pipeline de preprocesamiento con los datos de entrenamiento (puede tardar un poco)...")

pipeline_model = pipeline.fit(df_train)
print("Pipeline ajustado correctamente.")


Ajustando el pipeline de preprocesamiento con los datos de entrenamiento (puede tardar un poco)...


25/05/07 21:24:43 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 29:============================>                            (8 + 8) / 16]

Pipeline ajustado correctamente.


In [9]:
df_train

DataFrame[takeoff_time: double, timestamp: timestamp_ntz, icao: string, callsign: string, holding_point: string, runway: string, operator: string, turbulence_category: string, last_min_takeoffs: bigint, last_min_landings: bigint, last_event_turb_cat: string, time_since_last_event_seconds: bigint, time_before_holding_point: double, time_at_holding_point: double, hour: int, weekday: string, is_holiday: boolean, Z1: boolean, KA6: boolean, KA8: boolean, K3: boolean, K2: boolean, K1: boolean, Y1: boolean, Y2: boolean, Y3: boolean, Y7: boolean, Z6: boolean, Z4: boolean, Z2: boolean, Z3: boolean, LF: boolean, L1: boolean, LA: boolean, LB: boolean, LC: boolean, LD: boolean, LE: boolean, 36R_18L: boolean, 32R_14L: boolean, 36L_18R: boolean, 32L_14R: boolean, __index_level_0__: bigint, temperature_2m (°C): double, relative_humidity_2m (%): bigint, dew_point_2m (°C): double, apparent_temperature (°C): double, precipitation (mm): double, rain (mm): double, snowfall (cm): double, snow_depth (m): do

In [10]:
# --- Transformación de los Datos ---
# Ahora usamos el pipeline YA AJUSTADO (pipeline_model) para transformar AMBOS conjuntos de datos: train y test.
# Es crucial usar el MISMO modelo de pipeline (ajustado solo en train) para asegurar consistencia.
# Seleccionamos solo la columna objetivo y la nueva columna 'features' (vector escalado).
print("Transformando los datos de entrenamiento con el pipeline ajustado...")
df_train_prepared = pipeline_model.transform(df_train).select(TARGET_COL, "features")

print("Transformación completada.")

# Mostramos cómo quedaÇn los datos preparados (solo el target y el vector de features)
print("\nPrimeras filas de datos de entrenamiento preparados:")
# Usamos vector_to_array para una visualización más clara del vector, truncamos la salida
df_train_prepared.withColumn("features_array", vector_to_array("features")).show(5, truncate=True)

Transformando los datos de entrenamiento con el pipeline ajustado...
Transformación completada.

Primeras filas de datos de entrenamiento preparados:
+------------------+--------------------+--------------------+
|  log_takeoff_time|            features|      features_array|
+------------------+--------------------+--------------------+
|5.0369526024136295|(321,[0,16,21,228...|[1.0, 0.0, 0.0, 0...|
|5.0369526024136295|(321,[0,16,21,228...|[1.0, 0.0, 0.0, 0...|
| 5.043425116919247|(321,[0,16,21,228...|[1.0, 0.0, 0.0, 0...|
| 5.043425116919247|(321,[0,16,21,228...|[1.0, 0.0, 0.0, 0...|
| 5.049856007249537|(321,[0,16,21,228...|[1.0, 0.0, 0.0, 0...|
+------------------+--------------------+--------------------+
only showing top 5 rows



In [11]:
# --- 5. Conversión a Pandas/NumPy y Escalado del Target ---

# Convertimos los DataFrames de Spark preparados (con vectores de features) a DataFrames de Pandas.
# Esto es necesario porque Keras/TensorFlow trabaja con arrays de NumPy.
# La optimización con Arrow (habilitada antes) acelera esta conversión.
print("\nConvirtiendo DataFrames Spark preprocesados a Pandas (puede consumir memoria)...")
pdf_train = df_train_prepared.toPandas()
print("Conversión a Pandas completada.")

# Extraemos las features (vectores) y el target de los DataFrames de Pandas.
# np.stack convierte la columna de vectores densos de Spark (o arrays si usamos vector_to_array) en una matriz NumPy 2D.
X_train_raw = np.stack(pdf_train["features"].apply(lambda x: x.toArray()).values) # Aseguramos conversión a array denso
y_train_raw = pdf_train[TARGET_COL].values

# Mostramos las dimensiones de nuestros arrays NumPy resultantes.
print(f"Shape de X_train_raw (features): {X_train_raw.shape}, Shape de y_train_raw (target): {y_train_raw.shape}")

# Ahora escalamos el Target (takeoff_time) usando Sklearn MinMaxScaler.
# Es importante escalar el target en regresión para redes neuronales, a menudo al rango [0, 1].
# Creamos un escalador y lo AJUSTAMOS SÓLO con los datos de entrenamiento (y_train_raw).
print("\nEscalando la variable objetivo (takeoff_time) usando Sklearn MinMaxScaler...")
y_scaler = SklearnMinMaxScaler()
# fit_transform en train: ajusta y transforma
y_train_scaled = y_scaler.fit_transform(y_train_raw.reshape(-1, 1)).flatten()


Convirtiendo DataFrames Spark preprocesados a Pandas (puede consumir memoria)...


/home/bryan/pruebas/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)
                                                                                

Conversión a Pandas completada.
Shape de X_train_raw (features): (151524, 321), Shape de y_train_raw (target): (151524,)

Escalando la variable objetivo (takeoff_time) usando Sklearn MinMaxScaler...


In [12]:
# --- GUARDAR EL y_scaler AJUSTADO ---
import joblib # Necesitas importar joblib
y_scaler_path = './despegues/src/models/completo/y_scaler_trained_FINAL.gz' # Elige tu ruta y nombre
try:
    joblib.dump(y_scaler, y_scaler_path)
    print(f"y_scaler ajustado guardado exitosamente en: {y_scaler_path}")
except Exception as e:
    print(f"Error al guardar y_scaler: {e}")
# ------------------------------------

y_scaler ajustado guardado exitosamente en: ./despegues/src/models/completo/y_scaler_trained_FINAL.gz


In [12]:
# --- 6. Creación de Secuencias Temporales ---

# Definimos la longitud de las secuencias que usará nuestro modelo Transformer.
# El modelo mirará 'TIMESTEPS' pasos hacia atrás para predecir el siguiente.
TIMESTEPS = 10

# Definimos una función para convertir nuestros datos (features y target escalado) en secuencias.
# Para cada punto 'i', creamos una secuencia X de [i-TIMESTEPS:i] y un target Y en 'i'.
def create_sequences(X, y, timesteps):
    """Crea secuencias de datos para modelos temporales."""
    X_seq, y_seq = [], []
    print(f"Creando secuencias con timesteps={timesteps}...")
    # Aseguramos que haya suficientes datos para al menos una secuencia
    if len(X) <= timesteps:
        print(f"  ADVERTENCIA: No hay suficientes datos ({len(X)}) para crear secuencias de longitud {timesteps}.")
        return np.array([]).reshape(0, timesteps, X.shape[1]), np.array([]) # Devuelve arrays vacíos con forma correcta

    # Iteramos desde el primer punto posible que tiene 'timesteps' puntos antes
    for i in range(timesteps, len(X)): # Empezamos en 'timesteps' para tener historia
        X_seq.append(X[i-timesteps:i]) # La secuencia X va desde i-timesteps hasta i-1
        y_seq.append(y[i])             # El target Y es el valor en el instante i
    print(f"  Secuencias creadas: {len(X_seq)}")
    return np.array(X_seq), np.array(y_seq)

# Creamos las secuencias para entrenamiento
X_train_seq, y_train_seq = create_sequences(X_train_raw, y_train_scaled, TIMESTEPS)


# Mostramos las dimensiones de los arrays de secuencias resultantes.
# La forma esperada es (num_secuencias, TIMESTEPS, num_features) para X, y (num_secuencias,) para Y.
print(f"\nShape de X_train_seq: {X_train_seq.shape}, Shape de y_train_seq: {y_train_seq.shape}")

Creando secuencias con timesteps=10...
  Secuencias creadas: 151514

Shape de X_train_seq: (151514, 10, 321), Shape de y_train_seq: (151514,)


In [13]:
# --- Celda 7: Definición del Modelo Transformer (Simplificado) ---
import tensorflow as tf 

# Capa de Codificación Posicional (sin cambios)
class PositionalEncoding(tf.keras.layers.Layer):
    def call(self, x):
        seq_len = tf.shape(x)[1]
        embedding_dim = tf.shape(x)[2]
        pos = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
        i = tf.range(embedding_dim, dtype=tf.float32)[tf.newaxis, :]
        angle_rates = 1 / tf.pow(10000., (2 * (i // 2)) / tf.cast(embedding_dim, tf.float32))
        angle_rads = pos * angle_rates
        sines = tf.sin(angle_rads[:, 0::2])
        cosines = tf.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[:, :embedding_dim]
        return x + pos_encoding[tf.newaxis, :, :]

# --- Función para construir el modelo (ESTRUCTURA INTERNA SIMPLIFICADA) ---
def build_transformer_model(input_shape, units=64, dropout_rate=0.3, num_heads=4, key_dim=16, l2_reg=1e-4, learning_rate=0.001, num_blocks=1):
    """Construye el modelo Transformer SIMPLIFICADO para regresión."""
    print(f"Construyendo modelo SIMPLIFICADO con: units={units}, dropout={dropout_rate}, heads={num_heads}, key_dim={key_dim}, blocks={num_blocks}")
    inp = Input(shape=input_shape)

    # Capa Inicial
    x = PositionalEncoding()(inp)
    x = LayerNormalization(name='norm_post_posenc')(x)

    # Bloques Transformer Apilados
    for i in range(num_blocks):
        block_input = x # Entrada al bloque actual
        block_name_prefix = f'block_{i+1}_'

        # Sub-bloque Atención
        attn_output = MultiHeadAttention(
            num_heads=num_heads, key_dim=key_dim, name=f'{block_name_prefix}mha'
        )(query=x, value=x, key=x)
        attn_output = Dropout(dropout_rate, name=f'{block_name_prefix}dropout_attn')(attn_output)
        x = Add(name=f'{block_name_prefix}add_residual_1')([block_input, attn_output]) # 1ª Conexión Residual
        x = LayerNormalization(name=f'{block_name_prefix}norm_post_attn')(x)

        # Guardamos la entrada al sub-bloque FFN
        ffn_input = x # Esta es la salida normalizada del sub-bloque de atención
        
        # Un poco más estándar y a menudo mejor, introduce una expansión interna:
        # units_ffn_intermediate = units * 2 # O input_shape[-1] * 2, puedes experimentar
        units_ffn_intermediate = input_shape[-1] * 2 # Por ejemplo, expandir a 2 veces la dim de entrada
        ffn_output = Dense(units_ffn_intermediate, activation='gelu', kernel_regularizer=l2(l2_reg), name=f'{block_name_prefix}ffn_dense_expand')(ffn_input)
        ffn_output = Dense(input_shape[-1], kernel_regularizer=l2(l2_reg), name=f'{block_name_prefix}ffn_dense_contract')(ffn_output) # Proyectar de nuevo a la dimensión original
        ffn_output = Dropout(dropout_rate, name=f'{block_name_prefix}dropout_ffn')(ffn_output)
        
        # SEGUNDA Conexión Residual y LayerNorm (después de la FFN)
        x = Add(name=f'{block_name_prefix}add_residual_ffn')([ffn_input, ffn_output]) # Sumar la entrada al FFN (ffn_input) con su salida (ffn_output)
        x = LayerNormalization(name=f'{block_name_prefix}norm_post_ffn')(x)

    # Capa de Salida (después del último bloque)
    # El pooling ahora toma la salida del Dropout de la FFN del último bloque
    x_pooled = GlobalAveragePooling1D(name='global_avg_pooling')(x)
    # Mantenemos un Dropout después del pooling
    x_pooled = Dropout(dropout_rate, name='dropout_pooling')(x_pooled)

    # Capas Densas finales
    x_out = Dense(32, activation='gelu', kernel_regularizer=l2(l2_reg), name='dense_out_1')(x_pooled)

    # Salida lineal
    out = Dense(1, name='output_regression')(x_out)

    # Creamos y compilamos el modelo
    model = Model(inputs=inp, outputs=out)
    
    # Usa el schedule en Adam
    optimizer = Adam(learning_rate=learning_rate, clipvalue=1.0) # O clipnorm
    
    # Compila con el nuevo optimizador
    model.compile(optimizer=optimizer, loss='mae')
    return model

print("Función build_transformer_model SIMPLIFICADA definida.")

Función build_transformer_model SIMPLIFICADA definida.


In [ ]:
from tensorflow.keras.models import load_model
model = load_model("despegues/src/models/modeloTransformer3.keras")

In [14]:
# --- Celda 8: Construcción y Entrenamiento del Modelo (SIMPLIFICADO) ---

# Definimos la proporción para el conjunto de validación
VALIDATION_SPLIT_RATIO = 0.1 # Por ejemplo, el último 10% de las secuencias para validación

num_total_sequences = X_train_seq.shape[0]
num_val_sequences = int(num_total_sequences * VALIDATION_SPLIT_RATIO)
num_train_sequences = num_total_sequences - num_val_sequences

# Dividimos las secuencias 
X_train_final_seq = X_train_seq[:num_train_sequences]
y_train_final_seq = y_train_seq[:num_train_sequences]

X_val_seq = X_train_seq[num_train_sequences:]
y_val_seq = y_train_seq[num_train_sequences:]

print(f"\nDatos para entrenamiento final: X_train_final_seq shape: {X_train_final_seq.shape}, y_train_final_seq shape: {y_train_final_seq.shape}")
print(f"Datos para validación: X_val_seq shape: {X_val_seq.shape}, y_val_seq shape: {y_val_seq.shape}")


print("\nInstanciando el modelo Transformer SIMPLIFICADO...")
# Obtenemos la forma de entrada correcta (debería reflejar OHE si se aplicó)
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
print(f"Input shape para el modelo: {input_shape}")

# --- Hiperparámetros para esta prueba ---
# (Usamos los que pusiste, pero asegurando LR bajo y unidades razonables)
MODEL_UNITS = 64      # Mantenemos 64 como probaste
DROPOUT_RATE = 0.25   # Tasa de dropout
NUM_HEADS = 8        # Número de cabezas
# KEY_DIM se calcula basado en el input_shape real
KEY_DIM = input_shape[-1] // NUM_HEADS if input_shape[-1] else 16 # Asegura que key_dim sea válido
L2_REG = 1e-5         # Regularización L2
LEARNING_RATE = 0.0001 # Mantenemos el LR bajo que probaste
EPOCHS = 20          # Número máximo de épocas
BATCH_SIZE = 64       # Tamaño del lote
NUM_BLOCKS = 8        # Seguimos probando con 2 bloques

# Construimos el modelo SIMPLIFICADO
model = build_transformer_model(
    input_shape=input_shape,
    units=MODEL_UNITS,
    dropout_rate=DROPOUT_RATE,
    num_heads=NUM_HEADS,
    key_dim=KEY_DIM,
    l2_reg=L2_REG,
    learning_rate=LEARNING_RATE,
    num_blocks=NUM_BLOCKS # Pasamos el número de bloques
)

# Mostramos un resumen
model.summary()

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
print(f"\nEarlyStopping configurado con paciencia de {early_stop.patience} épocas.")

print("\n--- Iniciando el entrenamiento del modelo SIMPLIFICADO ---")
# Entrenamos
history = model.fit(
    X_train_final_seq, y_train_final_seq, # Usar los datos de entrenamiento divididos manualmente
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_val_seq, y_val_seq), # Pasar el conjunto de validación explícitamente
    callbacks=[early_stop],
    verbose=1,
    shuffle=True # shuffle=True aquí es bueno, barajará el ORDEN de las secuencias de X_train_final_seq en cada época
)
print("--- Entrenamiento finalizado ---")



Datos para entrenamiento final: X_train_final_seq shape: (136363, 10, 321), y_train_final_seq shape: (136363,)
Datos para validación: X_val_seq shape: (15151, 10, 321), y_val_seq shape: (15151,)

Instanciando el modelo Transformer SIMPLIFICADO...
Input shape para el modelo: (10, 321)
Construyendo modelo SIMPLIFICADO con: units=64, dropout=0.25, heads=8, key_dim=40, blocks=8


I0000 00:00:1746606822.167305   10921 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746606823.143094   10921 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746606823.143287   10921 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746606823.165118   10921 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1746606823.165475   10921 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 10, 321)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encoding │ (None, 10, 321)   │          0 │ input_layer[0][0] │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ norm_post_posenc    │ (None, 10, 321)   │        642 │ positional_encod… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_mha         │ (None, 10, 321)   │    412,161 │ norm_post_posenc… │
│ (MultiHeadAttentio… │                   │            │ norm_post_posenc… │
│                     │                   │            │ norm_post_posenc… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_dropout_at… │ (None, 10, 321)   │          0 │ block_1_mha[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_add_residu… │ (None, 10, 321)   │          0 │ norm_post_posenc… │
│ (Add)               │                   │            │ block_1_dropout_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_norm_post_… │ (None, 10, 321)   │        642 │ block_1_add_resi… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_ffn_dense_… │ (None, 10, 642)   │    206,724 │ block_1_norm_pos… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_ffn_dense_… │ (None, 10, 321)   │    206,403 │ block_1_ffn_dens… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_dropout_ffn │ (None, 10, 321)   │          0 │ block_1_ffn_dens… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_add_residu… │ (None, 10, 321)   │          0 │ block_1_norm_pos… │
│ (Add)               │                   │            │ block_1_dropout_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_norm_post_… │ (None, 10, 321)   │        642 │ block_1_add_resi… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_2_mha         │ (None, 10, 321)   │    412,161 │ block_1_norm_pos… │
│ (MultiHeadAttentio… │                   │            │ block_1_norm_pos… │
│                     │                   │            │ block_1_norm_pos… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_2_dropout_at… │ (None, 10, 321)   │          0 │ block_2_mha[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_2_add_residu… │ (None, 10, 321)   │          0 │ block_1_norm_pos… │
│ (Add)               │                   │            │ block_2_dropout_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_2_norm_post_… │ (None, 10, 321)   │        642 │ block_2_add_resi… │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 6,623,555 (25.27 MB)

...

2130/2131 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.2025

2025-05-07 10:39:17.929611: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_64', 8 bytes spill stores, 8 bytes spill loads

2025-05-07 10:39:18.416636: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_64', 316 bytes spill stores, 316 bytes spill loads

2025-05-07 10:39:19.519765: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_64', 40 bytes spill stores, 40 bytes spill loads

2025-05-07 10:39:19.984327: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_66', 628 bytes spill stores, 548 bytes spill loads

2025-05-07 10:39:20.628538: I external/local_xla/xla/stream_execut

2131/2131 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.2025

2025-05-07 10:41:12.538156: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1935', 4 bytes spill stores, 4 bytes spill loads

2025-05-07 10:41:13.825744: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1935', 8 bytes spill stores, 8 bytes spill loads



2131/2131 ━━━━━━━━━━━━━━━━━━━━ 444s 123ms/step - loss: 0.2024 - val_loss: 0.1669
Epoch 2/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 126s 59ms/step - loss: 0.1580 - val_loss: 0.1593
Epoch 3/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 83s 39ms/step - loss: 0.1440 - val_loss: 0.1526
Epoch 4/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - loss: 0.1286 - val_loss: 0.1505
Epoch 5/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 85s 40ms/step - loss: 0.1107 - val_loss: 0.1371
Epoch 6/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - loss: 0.0965 - val_loss: 0.1360
Epoch 7/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - loss: 0.0846 - val_loss: 0.1329
Epoch 8/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 83s 39ms/step - loss: 0.0755 - val_loss: 0.1336
Epoch 9/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 83s 39ms/step - loss: 0.0686 - val_loss: 0.1364
Epoch 10/20
2131/2131 ━━━━━━━━━━━━━━━━━━━━ 84s 39ms/step - loss: 0.0631 - val_loss: 0.1330
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 7.
--- Entrenamiento final

In [15]:
# --- Celda 9: Evaluación del Modelo (Continuación - Train Simplificado) ---
# Asegúrate de que las importaciones (numpy, sklearn.metrics) ya se hicieron
# y que las métricas de test (rmse_test, mae_test) se calcularon antes si quieres la comparación.

# --- Evaluación en el Conjunto de Entrenamiento (Simplificado) ---
print("\n--- Calculando Métricas de Entrenamiento ---")

# 1. Predecir en Train
# Asume que 'model', 'X_train_seq', 'y_train_seq', 'y_scaler' existen
y_pred_scaled_train = model.predict(X_train_seq)

# 2. Invertir Escalador MinMax (Obtiene valores en escala log original)
y_pred_log_train = y_scaler.inverse_transform(y_pred_scaled_train).flatten()
y_train_log_inv = y_scaler.inverse_transform(y_train_seq.reshape(-1, 1)).flatten()

# 3. Invertir Logaritmo (Obtiene valores en escala original - segundos)
print("Invirtiendo logaritmo para Train...")
y_pred_orig_train = np.expm1(y_pred_log_train)
y_train_orig = np.expm1(y_train_log_inv)

# 4. Calcular Métricas de Entrenamiento Directamente
print("Calculando métricas de entrenamiento...")
rmse_train = np.sqrt(mean_squared_error(y_train_orig, y_pred_orig_train))
mae_train = mean_absolute_error(y_train_orig, y_pred_orig_train)

print("\n--- Métricas en el Conjunto de Entrenamiento (Escala Original en Segundos) ---")
print(f"  Train RMSE: {rmse_train:.2f} segundos")
print(f"  Train MAE:  {mae_train:.2f} segundos")


--- Calculando Métricas de Entrenamiento ---
4735/4735 ━━━━━━━━━━━━━━━━━━━━ 41s 8ms/step
Invirtiendo logaritmo para Train...
Calculando métricas de entrenamiento...

--- Métricas en el Conjunto de Entrenamiento (Escala Original en Segundos) ---
  Train RMSE: 71.99 segundos
  Train MAE:  42.13 segundos


In [16]:
import importlib
import evaluator  # Importar el módulo por primera vez

importlib.reload(evaluator)  # Recargar el módulo

from evaluator import Evaluator

In [17]:
import pandas as pd
import numpy as np # Make sure numpy is imported here if needed in this script part

# Assuming df_train, pipeline_model, y_train_orig, mae_train, rmse_train are defined earlier

# This line seems unrelated to the fix for the KeyError and potentially confusing
# based on the flow below, so I'll comment it out or remove it if it's not used later.
# df_train_ev = df_train.toPandas() # <-- Remove or verify usage

print("Aplicando fitted_spark_pipeline a df_train para preparar la alineación...")
# Asegúrate de que df_train está ordenado por Timestamp si es necesario para la creación de secuencias
# Si tu df_train ya está ordenado por Timestamp, no necesitas este paso extra.
# Si no, ordénalo: df_train = df_train.orderBy("Timestamp") # Reemplaza "Timestamp" con el nombre real de tu columna de tiempo

df_processed_train_spark = pipeline_model.transform(df_train)
print(f"Después de aplicar fitted_spark_pipeline: df_processed_train_spark.count() = {df_processed_train_spark.count()}")
# df_processed_train_spark.printSchema() # Opcional: verificar esquema

# --- Paso 2: Convertir el DataFrame Spark procesado a Pandas ---
# Esto es necesario para poder alinear fácilmente las predicciones (arrays NumPy)
# con las filas correctas que fueron los objetivos de las secuencias.
print("Convirtiendo DataFrame Spark procesado a Pandas...")
df_processed_train_pandas = df_processed_train_spark.toPandas()
print(f"DataFrame Pandas procesado creado con {len(df_processed_train_pandas)} filas.")

# --- Paso 3: Identificar las filas en el DataFrame de Pandas que corresponden a los puntos objetivo de las secuencias ---
# Las predicciones (y_pred_orig_train) corresponden al último paso de tiempo de cada secuencia.
# Si la primera secuencia usa los índices 0 a SEQUENCE_LENGTH-1, su objetivo es el índice SEQUENCE_LENGTH-1 en el DataFrame original.
# La última secuencia usa los últimos SEQUENCE_LENGTH puntos, su objetivo es el último índice.
# Por lo tanto, las filas objetivo en el DataFrame de Pandas son desde el índice SEQUENCE_LENGTH - 1 hasta el final.
start_index_for_targets = 10 # Assuming SEQUENCE_LENGTH is 10 based on this line
# Seleccionamos las filas desde el índice de inicio hasta el final
df_target_rows_pandas = df_processed_train_pandas.iloc[start_index_for_targets:].copy()
print(f"Filas objetivo seleccionadas en Pandas DataFrame: {len(df_target_rows_pandas)} filas.")

# --- Paso 4: Añadir las columnas de valores reales y predichos en escala original ---
# Los arrays (y_train_orig, y_pred_orig_train) ya están alineados con los puntos objetivo.
# Asegúrate de que la longitud de los arrays coincide con el número de filas objetivo seleccionadas.
if len(df_target_rows_pandas) == len(y_train_orig) and len(df_target_rows_pandas) == len(y_pred_orig_train):
    # Add the 'prediction' column
    df_target_rows_pandas['prediction'] = y_pred_orig_train
    print("Column 'prediction' added to the Pandas DataFrame.")

    # *** FIX 1: Add the 'takeoff_time' column with the actual target values ***
    # The Evaluator class expects a column named 'takeoff_time'.
    # The original target values are in y_train_orig.
    df_target_rows_pandas['takeoff_time'] = y_train_orig
    print("Column 'takeoff_time' (from y_train_orig) added to the Pandas DataFrame.")

    # The 'actual_takeoff_time_s' column is not strictly necessary for the Evaluator
    # but can be kept if needed elsewhere. For the fix, ensuring 'takeoff_time' exists is key.
    # df_target_rows_pandas['actual_takeoff_time_s'] = y_train_orig # Optional: Keep if needed

    print("\n8. Initializing Evaluator...")
    # *** FIX 2: Pass the correct DataFrame (df_target_rows_pandas) to the Evaluator ***
    # Removed the incorrect line: df_train_evaluated = df_train_evaluated.toPandas()
    ev = Evaluator(df_target_rows_pandas, "Transformer (Train Evaluation)", mae_train, rmse_train)
    print("   Generando reporte...")
    report = ev.getReport()
    print("   Reporte del Evaluator:")
    print(report)
    print("   Generando evaluación visual...")
    ev.visualEvaluation()

else:
    # This warning about length mismatch indicates an issue with your data preparation/alignment
    # before this point. The Evaluator cannot be initialized correctly if the lengths don't match.
    print(f"Advertencia: El número de filas objetivo en el DataFrame de Pandas ({len(df_target_rows_pandas)}) no coincide con la longitud de los arrays de target/predicciones ({len(y_train_orig)}). No se añadieron las columnas de evaluación. Cannot initialize Evaluator.")
    # You might want to exit or raise an error here instead of just printing a warning
    # if correctly aligned data is critical.

Aplicando fitted_spark_pipeline a df_train para preparar la alineación...
Después de aplicar fitted_spark_pipeline: df_processed_train_spark.count() = 151524
Convirtiendo DataFrame Spark procesado a Pandas...


/home/bryan/pruebas/.venv/lib/python3.11/site-packages/pyspark/sql/pandas/conversion.py:111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 4.0.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)
                                                                                

DataFrame Pandas procesado creado con 151524 filas.
Filas objetivo seleccionadas en Pandas DataFrame: 151514 filas.
Column 'prediction' added to the Pandas DataFrame.
Column 'takeoff_time' (from y_train_orig) added to the Pandas DataFrame.

8. Initializing Evaluator...
   Generando reporte...
   Reporte del Evaluator:
{'global': {'mae': 42.13151751277467, 'rmse': 71.99313262332993, 'mse': 5183.011144920372, 'r2': 0.5695391876374878, 'mape': 21.142244627918743}, 'by_runway': {'32L/14R': {'mae': 76.59706911537658, 'rmse': 118.03405121747524}, '32R/14L': {'mae': 38.427188255385374, 'rmse': 72.18797690391855}, '36L/18R': {'mae': 46.62828793670503, 'rmse': 73.23590084395538}, '36R/18L': {'mae': 29.780862913069555, 'rmse': 56.1133327667314}}, 'by_holding_point': {'K1': {'mae': 64.12744851406876, 'rmse': 121.39279095471898}, 'K2': {'mae': 33.52177985125911, 'rmse': 58.17047562830251}, 'K3': {'mae': 82.1287760053362, 'rmse': 169.78428821157718}, 'LA': {'mae': 76.25846497984567, 'rmse': 127.767

[2025-05-07 11:22:46,120] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/bryan/pruebas/.venv/lib/python3.11/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bryan/pruebas/.venv/lib/python3.11/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bryan/pruebas/.venv/lib/python3.11/site-packages/dash/dash.py", line 1414, in dispatch
    ctx.run(
  File "/home/bryan/pruebas/.venv/lib/python3.11/site-packages/dash/_callback.py", line 536, in add_context
    raise err
  File "/home/bryan/pruebas/.venv/lib/python3.11/site-packages/dash/_callback.py", line 525, in add_context
    output_value = _invoke_callback(func, *func_args, **func_kwar

In [30]:
model_save_directory = 'despegues/src/models'
model_filename = 'modeloTransformer3.keras'

# Construye la ruta completa del archivo
full_model_path = os.path.join(model_save_directory, model_filename)


# Asegúrate de que el directorio exista
# os.makedirs() solo creará directorios, no el archivo final
try:
    os.makedirs(model_save_directory, exist_ok=True)
    print(f"Directorio de guardado asegurado: {model_save_directory}")
except Exception as e:
    print(f"Error al crear el directorio de guardado: {e}")
    # Si falla la creación del directorio, no intentes guardar el modelo
    full_model_path = None # Establecer a None para evitar el intento de guardado


# Guarda el modelo en formato nativo de Keras (.keras)
# Esto guarda la arquitectura, los pesos y la configuración de entrenamiento.
if full_model_path:
    print(f"Guardando el modelo en: {full_model_path}")
    try:
        model.save(full_model_path)
        print("Modelo guardado exitosamente.")
    except Exception as e:
        print(f"Error al guardar el modelo: {e}")

Directorio de guardado asegurado: despegues/src/models
Guardando el modelo en: despegues/src/models/modeloTransformer3.keras
Modelo guardado exitosamente.


En el método de evaluación anterior, dividíamos el df_test usando el valor real y final de log_takeoff_time. Esto significaba que para decidir si usar model_low o model_high para una secuencia de prueba, estábamos mirando el resultado final (el tiempo total de espera), algo que no sabríamos en el momento de hacer la predicción real. Esto inflaba artificialmente los resultados porque siempre dirigíamos la secuencia al modelo "correcto". 

Correccion: No usar ese umbral ni columna en el test. Derivar unos u otros mensajes del test desde otro umbral que no use ninguna variable de salida.